# Lascar Example with CW Traces

Capturing is the same as usual. We'll capture a rather large amount of trace (3000) to show off the speed of lascar:

In [ ]:
%run "Helper_Scripts/CWLite_Connect.ipynb"

In [ ]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

In [ ]:
# uncomment based on your target
#%run "Helper_Scripts/Program_XMEGA.ipynb"
%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/No_Programmer.ipynb"
fw_path = "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-cwlitearm.hex"

In [ ]:
# program the target
program_target(scope, fw_path)

In [ ]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time

ktp = AcqKeyTextPattern_Basic(target=target)

traces = []
textin = []
keys = []
N = 3000  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    textin.append(text)
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    traces.append(scope.getLastTrace())

#Convert traces to numpy arrays
trace_array = np.asarray(traces)  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(textin)
known_keys = np.asarray(keys)  # for fixed key, these keys are all the same

In [127]:
import chipwhisperer as cw
import numpy as np

project = cw.openProject("../stm32f415_lab.cwp")

start_point = 1312
end_point = 1350

tm = project.traceManager()

trace_array = np.zeros( (tm.numTraces(), end_point - start_point))
textin_array = np.zeros( (tm.numTraces(), len(tm.getTextin(0))), dtype="uint8" )
textout_array = np.zeros( (tm.numTraces(), len(tm.getTextout(0))), dtype="uint8" )

print ("Copying %d traces of %d samples into memory" % (tm.numTraces(), tm.numPoints()))
for n in range(0, tm.numTraces()):
    trace_array[n] = tm.getTrace(n)[start_point:end_point]
    textin_array[n] = tm.getTextin(n)
    textout_array[n] = tm.getTextout(n)

Copying 5000 traces of 2000 samples into memory


## Running Lascar on Traces

The first thing we will do is select a leakage function. The following is a few examples - the first is the standard sboxHW function, the next two are common for hardware crypto.

In [128]:
from lascar import *
from lascar.tools.aes import sbox, inv_sbox

#The following leakage models copied from /chipwhisperer/analyzer/attacks/models/AES128_8bit.py and
# massaged into Lascar Version

def selection_function_sboxHW(byte):
    # selection_with_guess function must take 2 arguments: value and guess
    def selection_with_guess(value, guess):
        return hamming(sbox[value[byte] ^ guess])
    return selection_with_guess

def selection_function_sboxInOutHD(byte):
    # selection_with_guess function must take 2 arguments: value and guess
    def selection_with_guess(value, guess):
        return hamming(sbox[value[byte] ^ guess] ^ value[byte])
    return selection_with_guess

def selection_function_lastroundHD(byte):
    # selection_with_guess function must take 2 arguments: value and guess
    def selection_with_guess(value, guess):
        INVSHIFT_undo = [0, 5, 10, 15, 4, 9, 14, 3, 8, 13, 2, 7, 12, 1, 6, 11]
        st10 = value[INVSHIFT_undo[byte]]
        st9 = inv_sbox[value[byte] ^ guess]
        return hamming(st9 ^ st10)
    return selection_with_guess

Now, we need to make the engines. This requires us to select one of the above leakage functions and create a large array of them. Edit the following to change the leakage function you wish to use.

In [129]:
from chipwhisperer.analyzer.attacks.models.AES128_8bit import LastroundStateDiff
#Adjust this if needed - will ensure correct key/PGE highlighting is done!
highlight_key = LastroundStateDiff().processKnownKey(project.traceManager().getKnownKey(0))
#highlight_key = project.traceManager().getKnownKey(0)

#Adjust this for actual attack used!
guess_range = range(256)
cpa_engines = [CpaEngine("cpa_%02d" % i, selection_function_lastroundHD(i), guess_range) for i in range(16)]

In [130]:
#This is alist of cpa engines now. See for example:
#help(cpa_engines[0])

Finally run ONE of the following - the difference is the container either has the textin or textout. The correct setup depends on your leakage function assumptions.

In [119]:
#Leakage models using plaintext (such as SBox output) require textin
containter_textin =  TraceBatchContainer(trace_array, textin_array)
session = Session(containter_textin, engines=cpa_engines).run(batch_size=50)

2018-11-24 20:04:04,184 - lascar.session - INFO - Session Session: 5000 traces, 18 engines, batch_size=50, leakage_shape=(5,)
INFO:lascar.session:Session Session: 5000 traces, 18 engines, batch_size=50, leakage_shape=(5,)
Session |100%||5000 trc/5000 | (18 engines, batch_size=50, leakage_shape=(5,)) |Time:  0:00:13


In [131]:
#Leakage models using ciphertext (such as lastroundHD) require textout
containter_textout =  TraceBatchContainer(trace_array, textout_array)
session = Session(containter_textin, engines=cpa_engines).run(batch_size=50)

2018-11-24 20:05:47,609 - lascar.session - INFO - Session Session: 5000 traces, 18 engines, batch_size=50, leakage_shape=(5,)
INFO:lascar.session:Session Session: 5000 traces, 18 engines, batch_size=50, leakage_shape=(5,)
Session |100%||5000 trc/5000 | (18 engines, batch_size=50, leakage_shape=(5,)) |Time:  0:00:12


In [132]:
from IPython.display import clear_output
import numpy as np
import chipwhisperer as cw

from chipwhisperer.analyzer.attacks._stats import DataTypeDiffs

key = project.traceManager().getKnownKey(0)

class LascarCWAttacks(object):
    
    def __init__(self, cpa_engines, highlight_key=None):  
        dt = DataTypeDiffs()
        for i in range(len(cpa_engines)):
            results = cpa_engines[i].finalize()
            dt.updateSubkey(i, results)

        self.dt = dt
        self.hlk = highlight_key

    def display_pge(self):
        cb = cw.getJupyterCallback(self)
        cb()
        
    def getStatistics(self):
        """CW Interfae Function"""
        return self.dt
    
    def knownKey(self):
        """CW Interface Function"""
        if self.hlk is None: return [0]*16
        
        return self.hlk
    
    def getReportingInterval(self):
        """CW Interface Function"""
        return 0
    
results = LascarCWAttacks(cpa_engines, highlight_key)
results.display_pge()
    

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
PGE=,0,10,0,0,21,0,0,0,1,0,0,0,0,5,0,0
0,D00.079,7D0.055,F90.079,A80.059,220.057,EE0.065,250.062,890.059,AD0.054,3F0.069,0C0.063,C80.073,B60.083,A20.063,0C0.050,A60.071
1,2C0.054,580.048,F80.054,080.053,DF0.056,F60.048,A10.045,990.048,E10.050,DA0.052,CA0.048,0E0.047,C90.054,740.045,C30.048,580.052
2,880.048,E90.047,F00.043,FC0.042,5B0.055,920.047,4B0.045,140.047,F90.050,9F0.046,B00.045,570.044,520.053,D00.041,7B0.046,E00.047
3,110.048,FD0.045,210.043,E50.042,EA0.050,1B0.045,C30.044,920.045,2D0.050,4F0.042,B30.045,FF0.043,540.049,D70.040,800.044,4F0.046
4,950.046,AA0.045,FE0.040,DB0.042,ED0.047,1F0.042,F90.043,7F0.042,7D0.048,110.041,780.040,E50.042,0C0.048,440.039,F40.043,F10.044


## Unfinished Business

In [133]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
for i in range(16):
    results = cpa_engines[i].finalize()
    xrange = range(len(results[0x2B]))
    guess = abs(results).max(1).argmax()
    print("Best Guess is {:02X} (Corr = {})".format(guess, abs(results).max()))
    p.line(xrange, results[guess])
    
show(p)

Loading BokehJS ...

Best Guess is D0 (Corr = 0.07879607795290498)
Best Guess is 7D (Corr = 0.05525528778135184)
Best Guess is F9 (Corr = 0.07915459080709715)
Best Guess is A8 (Corr = 0.0593940064352325)
Best Guess is 22 (Corr = 0.05723682904145104)
Best Guess is EE (Corr = 0.06474946470546873)
Best Guess is 25 (Corr = 0.06175697418310108)
Best Guess is 89 (Corr = 0.058841606504510216)
Best Guess is AD (Corr = 0.05414313251816959)
Best Guess is 3F (Corr = 0.06893547285405739)
Best Guess is 0C (Corr = 0.06322474246191614)
Best Guess is C8 (Corr = 0.07268784872864047)
Best Guess is B6 (Corr = 0.08299322506283903)
Best Guess is A2 (Corr = 0.06326720229582192)
Best Guess is 0C (Corr = 0.05010140000298766)
Best Guess is A6 (Corr = 0.07074699432772671)


In [ ]:
print(results)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
xrange = range(len(results[0x2B]))
print(xrange)
print(len(results))
p = figure()
#p.line(xrange, traces[5],line_color='red')
p.line(xrange, results[0x3C])
show(p)

In [ ]:
print(mycontainer.values)

In [ ]:
from lascar.output.parse_results import apply_parse
import pandas as pd
from IPython.display import clear_output
class JupyterOutputMethod(OutputMethod):
    def __init__(self, *engines):
        OutputMethod.__init__(self, engines)
        self.parsed_subkeys = 0
        self.results_list = [0] * 16
    
    def _update(self, engine, results):
        engine.output_parser_mode = "argmax"
        results_parsed = apply_parse(engine, abs(results))
        display(self.parsed_subkeys)
        if results_parsed is None:
            return
        idx = int(cpa_engine.name[-2:])
        self.results_list[idx] = results_parsed
        self.parsed_subkeys += 1
        if self.parsed_subkeys > 15:
            df = pd.DataFrame(self.results_list)
            fd = df.transpose()
            def formatter(stats):
                return str("{}".format(stats))
            #clear_output(wait=True)
            display(fd.head().style.format(formatter))
            self.parsed_subkeys = 0
        
    def _finalize(self):
        pass